In [ ]:
from sklearn.neighbors import KNeighborsClassifier
import cv2
import pickle
import numpy as np
import os
import csv
import time
from datetime import datetime
from win32com.client import Dispatch

def speak(text):
    speaker = Dispatch("SAPI.SpVoice")
    speaker.Speak(text)

# Initialize video capture and face detection
video = cv2.VideoCapture(0)
facedetect = cv2.CascadeClassifier('C:/Users/pc/Downloads/haarcascade_frontalface_default.xml')

# Load pre-trained face data and labels
try:
    with open('C:/Users/pc/Downloads/names.pkl', 'rb') as f:
        LABELS = pickle.load(f)
    with open('C:/Users/pc/Downloads/faces_data.pkl', 'rb') as f:
        FACES = pickle.load(f)
except FileNotFoundError as e:
    print("Error loading files:", e)
    exit(1)

# Check and align shapes of FACES and LABELS
if FACES.shape[0] != len(LABELS):
    min_samples = min(FACES.shape[0], len(LABELS))
    FACES = FACES[:min_samples]
    LABELS = LABELS[:min_samples]
    print(f"Warning: Mismatched sample counts. Adjusted to {min_samples} samples.")

print('Shape of Faces matrix --> ', FACES.shape)
print('Length of Labels --> ', len(LABELS))

# Train KNN model
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(FACES, LABELS)

# Load background image
imgBackground = cv2.imread("C:/Users/pc/Downloads/background.png")

# Define CSV header for attendance
COL_NAMES = ['NAME', 'TIME']

# Create Attendance directory if not exists
if not os.path.exists("Attendance"):
    os.makedirs("Attendance")

while True:
    ret, frame = video.read()
    if not ret:
        print("Failed to grab frame.")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = facedetect.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        crop_img = frame[y:y+h, x:x+w]
        resized_img = cv2.resize(crop_img, (50, 50)).flatten().reshape(1, -1)
        
        # Predict the face label
        output = knn.predict(resized_img)
        
        # Get current timestamp
        ts = time.time()
        date = datetime.fromtimestamp(ts).strftime("%d-%m-%Y")
        timestamp = datetime.fromtimestamp(ts).strftime("%H:%M-%S")
        attendance_file = f"Attendance/Attendance_{date}.csv"
        
        # Draw bounding box and label
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 1)
        cv2.rectangle(frame, (x, y-40), (x+w, y), (50, 50, 255), -1)
        cv2.putText(frame, str(output[0]), (x, y-15), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 1)
        
        # Prepare attendance entry
        attendance = [str(output[0]), timestamp]

    # Display frame on background
    imgBackground[162:162 + 480, 55:55 + 640] = frame
    cv2.imshow("Frame", imgBackground)

    k = cv2.waitKey(1)
    
    # Save attendance on pressing 'o'
    if k == ord('o'):
        speak("Attendance Taken.")
        time.sleep(5)
        
        # Check if file exists, if not create with headers
        if os.path.isfile(attendance_file):
            with open(attendance_file, "a", newline='') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow(attendance)
        else:
            with open(attendance_file, "w", newline='') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow(COL_NAMES)
                writer.writerow(attendance)

    # Quit on pressing 'q'
    if k == ord('q'):
        break

# Release resources
video.release()
cv2.destroyAllWindows()